# Dropbox
- Store data on remote servers.

## Functional
- Users can upload/download photos/videos from any device.
- Users can share files/folders with other users.
- Automatic sync betweeen devices.
- Support large files up to 1GB.
- ACID required.
- Support offline editing.

## Non-functional

## Extended
- Support snapshotting data.

## Design
- Read/write will be huge.
- Read/write raio will be similar.
- Files should be stored in small chucks. (assume 4MB)
- Transfer updated chucks only.
- Keep file metadata on client to save traffics.

<img src="img/dropbox2.png" style="width:600px;height:400px;">

### Client
- Internal metadata DB: keeps metadata in the client to save round-trips to update remote metadata.
- Chunker: splits files into small pieces. Also, reconstruct a file from its chunks. 
- Watcher: monitors workspace and notify Indexer of any user actions. Listens to changes in other clients.
    - When listening to changes, use HTTP long polling where servers hold the client request open and respond when new information is available. 
- Indexer: processes events from Watcher and updates internal metadata DB. Also, updates remote DB via talking to remote sync service.

### Metadata DB
- Choose relational DB like MySQL for ACID.
- Information such as chucks, files, user, devices, workspace.

### Sync service
- Sync client metadata DB and remote metadata DB.
- To tranfer metadata, divide files into 4MB chucks and transfer modified chucks only. 

### Message queue
- Handles communication between clients and sync service.
- One global queue (Request Queue) to handle client request to update metadata DB.
- One queue for each client (Response Queue) send update message back to client.

### Block storage
- Stores chunks of files.
- Clients directly interact with storage.

### File processing
- Client A uploads chucks.
- Client A updates metadata.
- Client A gets ack and client B and C get the notification.
- Client B and C download metadata changes and chucks.

### Data deduplication
- For each incoming chuck, calculate its hash and compare it to hashes of all exising chucks.

### Security
- Store file permission in metadata DB to configure who can read/write.

<img src="img/dropbox3.png" style="width:600px;height:400px;">

## Capacity

### Assume 
- 500M total users, 100 daily active users.
- Each user connects from 3 different devices.
- Each user has 200 files/photos, so there we have 100 billion total files.
- Average file size is 100KB. We have 100B * 100KB = 10PB
- 1M active connections per minute.

## Data partitioning
- For partitioning metadata, use consistent hashing.

## Caching
- On block storage to deal with hot chucks.
- On block servers to see if chunks are already in cache.
    - Use LRU eviction policy.

## Load balancing
- Between clients and block servers.
- Between clients and metadata servers.
- Round robin.